# DSFB Course Project: Predicting IPO Share Price

![](https://img.etimg.com/thumb/height-480,width-640,msid-64038320,imgsize-108012/stock-market2-getty-images.jpg)

image source : https://img.etimg.com/thumb/height-480,width-640,msid-64038320,imgsize-108012/stock-market2-getty-images.jpg

## Introduction

An Initial Public Offering (IPO) is the process by which a private company becomes publicly traded on a stock exchange. The IPO company offers its shares to public investors in exchange of capital for sustaining expansion and growth. For this reason, IPOs are often issued by small or young companies, but they can also be done by large  companies looking to become publicly traded. During an IPO, the company obtains the assistance of an investment bank (underwriter), which helps determine the type, amount and price of the shares being offered. Decisions about the offering price are particularly important to avoid incurring excessive costs and maximize the capital received in the IPO. However at the end of the first trading day, price of each share can change due to market dynamics, which can lead to a price higher or lower than the offering one.

During an Initial Public Offering (IPO), the firm’s management have to disclose all relevant information about their business in a filing with the government called the "IPO Prospectus." Although there might be concerns about the public disclosure of sensitive information in the Prospectus that can help competitors, firms are encouraged to be as transparent as possible in order to avoid future litigation (lawsuits). A key textual field from the prospectus is:

__Risk_Factors__: Firms have to disclose all relevant information about internal or external risk factors that might affect future business performances. This information is contained in the “Risk Factors” section of the IPO prospectus. 

The key pricing variables are:

__Offering_Price__: the price at which a company sells its shares to investors.

__Num_Shares__: the total number of outstanding shares.

__Closing_Price__: (at the end of the first day of training) price at which shares trade in the open market, measured at the end of the first day of trading.

In this project you are provided with IPO data of different firms that are collected from different sources. You can find the dataset under project directory in the course git repository under the name of *ipo.xlsx*. The description of other variables can be found in *variable_description.xlsx*.

This Notebook will be presented as follow :

# Part 0 : Pre Processing
#####  libraries & useful functions
### 0.1 Import Dataset
### 0.2 Pre-processsing of Special Features
### 0.3 Pre-processing of Categorical Features
### 0.4 Pre-processing of Ordinal Features
### 0.5 Pre-processing of Dates Features
### 0.6 Pre-processing of Numerical Features
### 0.7 Pre-processing of Textual Features
### 0.8 Target
### 0.9 Feature Reduction
        1 Correlation with target
        2 Correlation between features
# Part 1 :
# Part 2 :
# Part 3 :
# Part 4 :

# Part 0 : Pre-Processing

##### Libraries & Useful functions  

In [14]:
import seaborn as sns
import re as re
import nltk
import datetime as dt

from datetime import datetime, timedelta
from sklearn.base import TransformerMixin
from sklearn.metrics import roc_curve
from nltk.corpus import stopwords 

In [15]:
from preprocessing_functions import *

## 0.1 Import Dataset 

##### Import,  explore dataset and check missing values

In [16]:
#read dataset
ipo = pd.read_excel('ipo.xlsx')

#### Declaration of categorization table to use in the data preprocessing

##### Numerical Columns

In [17]:
num_col = ['amd_nbr','round_tot','mgt_fee', 'gross_spread', 'min_round_vexp','avg_round_vexp', 'max_round_vexp',\
           'min_firm_amt_vexp', 'avg_firm_amt_vexp', 'max_firm_amt_vexp',\
          'min_fund_amt_vexp', 'max_fund_amt_vexp','description_numeric', 'rate']

##### Categorical Columns

In [18]:
cat_col = ['exch', 'mgrs_role', 'mgrs','all_sic','auditor', 'city','description','ht_ind', 'ht_ind_gr','industry', 'ind_group', 'issuer',\
          'veic_descr','legal', 'naic_primary','naic_decr', 'public_descr', 'lockup_flag','sic_main', 'nation', 'lbo', 'prim_naic', 'prim_uop', 'pe_backed', 'shs_out_after',\
          'state', 'uop', 'vc', 'zip','description_cat' ]

##### Columns to Drop

In [19]:
columns_to_drop = []
columns_to_drop.append('ID')

In [20]:
#Check if duplicates
ipo[ipo.astype(str).duplicated()==True]

,ID,Closing_Price,Offering_Price,Risk_Factors,mgt_fee,pctchg_dj_1,pctchg_hp,pctchg_lp,pctchg_mp,pctchg_nasdaq_1,...,num_funds_vexp,min_round_vexp,avg_round_vexp,max_round_vexp,min_firm_amt_vexp,avg_firm_amt_vexp,max_firm_amt_vexp,min_fund_amt_vexp,max_fund_amt_vexp,Num_Shares


In [21]:
#Number of actual features
len(ipo.columns)

154

In [22]:
# check missing values
nb_missing_values = sum(map(any, ipo.isnull()))
nb_missing_values

154

In [30]:
#Count missing 
features_with_nan = len(ipo) - ipo.count()
features_with_nan=features_with_nan[features_with_nan!=0]
nan_percentage=(features_with_nan/len(ipo)).to_frame('percentage').reset_index().rename(columns={'index':'feature'})

In [31]:
features_nan_percentage = nan_percentage[nan_percentage.percentage>0.5]
features_nan_percentage

,feature,percentage
10,pct_int_shs,0.831667
25,bvps_bef_offer,0.582333
42,int_aft,0.604333
43,int_bef,0.602333
44,intern)shs,0.831667
62,pb_value,0.583667


In [32]:
#Drop column with more than 80% of missing values
list_to_append = features_nan_percentage.feature.values
for col in list_to_append:
    columns_to_drop.append(col)

In [33]:
#Check type of values
ipo.dtypes.unique()

array([dtype('O'), dtype('float64'), dtype('<M8[ns]'), dtype('int64')],
      dtype=object)

In [34]:
#Devide dataframe in function of type.
ints = ipo.select_dtypes(include='int64')
floats = ipo.select_dtypes(include='float64')
objects = ipo.select_dtypes(include='object')
timestamps = ipo.select_dtypes(include='M8[ns]')

In [35]:
date_col = ['amd_date', 'lockup_date', 'lockup_days']

## 0.2 Pre-processsing of Special Features


In [36]:
# make a copy of the df
ipo_processing = ipo.copy()

### Description feature

We split it into 2 one numerical and second categorical

In [37]:
ipo_processing['description'].head(2)

0    5,000,000.0 Common Shares
1    6,898,541.0 Common Shares
Name: description, dtype: object

In [38]:
# delete the ',' in the features description
ipo_processing['description'] = ipo['description'].str.replace(',','',regex=False)
# split description into 2 string
split_description = ipo_processing.description.str.split('.0 ')
# create desctiption_numeric as the nb written in description 
ipo_processing['description_numeric']= split_description.apply(lambda x: int(x[0]))
# create description cat as the category of the number of shares
ipo_processing['description_cat'] =  split_description.apply(lambda x: x[1])

ipo_processing['description_numeric'].unique()

array([ 5000000,  6898541,  2700000, ...,  2816750, 18779865,  5781126],
      dtype=int64)

In [39]:
# drop description
columns_to_drop.append('description')

### all_sic : COMMENT THE CELL BY EXPLAINING WHATS HAPPENING INSIDE

In [40]:
ipo_processing.all_sic.unique()

array(['8093', '6282/6722/6726/6799/6719/6351', '5945/5199/6719/5999',
       ..., '7372/7319/7311', '3111/3172', '6519/6531/6162'], dtype=object)

We will try to reduce this number by a 2 digit categories for all_sic

In [41]:
ipo_processing.all_sic = ipo.all_sic.str.split('/')

ipo_processing.all_sic = ipo_processing.all_sic.apply(lambda x: [a[:2] for a in x])

We will use this file to reduce the number of categories:

In [42]:
all_sic_mapping = pd.read_excel('all_sic_mapping.xlsx',delimiter = ',')

In [43]:
all_sic_mapping.head()

,range_sic,cat
0,0100-0999,"Agriculture, Forestry and Fishing"
1,1000-1499,Mining
2,1500-1799,Construction
3,1800-1999,not used
4,2000-3999,Manufacturing


In [44]:
## Cleaning file:
all_sic_mapping.range_sic = all_sic_mapping.range_sic.str.split('-')

all_sic_mapping.range_sic= all_sic_mapping.range_sic.apply(lambda x: [int(a[:2]) for a in x])

all_sic_mapping.range_sic = all_sic_mapping.range_sic.apply(lambda x: np.arange(x[0],x[1]+1))

In [45]:
all_sic_mapping.head()

,range_sic,cat
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9]","Agriculture, Forestry and Fishing"
1,"[10, 11, 12, 13, 14]",Mining
2,"[15, 16, 17]",Construction
3,"[18, 19]",not used
4,"[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3...",Manufacturing


In [46]:
## Map all_sic values to new ones
splitted = all_sic_mapping.range_sic.apply(pd.Series).stack().reset_index(level = 1,drop = True).to_frame('range')

merged = pd.merge(splitted,all_sic_mapping,left_on = splitted.index, right_on=  all_sic_mapping.index,how = 'left')

merged_2 = merged[['range','cat']]

merged_2.range = merged_2.range.astype(int)

merged_2.set_index('range',inplace = True)


ipo_processing.all_sic = ipo_processing.all_sic.apply(lambda x : [merged_2.loc[int(a)] if (int(a) in merged_2.index.values) else int(a) for a in x ] )

ipo_processing.all_sic = ipo_processing.all_sic.apply(lambda x: [item for sublist in x for item in sublist])

c:\users\marie\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [47]:
ipo_processing = pd.concat([ipo_processing,pd.get_dummies(ipo_processing.all_sic.apply(pd.Series).stack()).sum(level=0)],axis = 1)
columns_to_drop.append('all_sic')

### zip : done
We will use this file 'us_postal_codes.csv' to get cities and state to replace where there is missing values

In [48]:
ipo['state'].isna().sum()

0

In [49]:
len(ipo['zip'].unique())

1507

In [50]:
us_zip = pd.read_csv('us_postal_codes.csv', sep=',')
us_zip.rename(columns={'Zip Code': 'zip', 'State':'state'}, inplace=True)
# Drop infos that we don't want
us_zip = us_zip.drop(columns=['Place Name', 'State Abbreviation', 'County', 'Latitude', 'Longitude'])
us_zip.head()

,zip,state
0,501,New York
1,544,New York
2,1001,Massachusetts
3,1002,Massachusetts
4,1003,Massachusetts


In [51]:
#Merge State in function of the ZIP code
ipo_processing.zip = ipo.zip.apply(lambda x: str(x)[:5])
ipo_processing.zip = ipo_processing.zip.apply(lambda x: int(x) if (str.isdigit(x)) else 0 )

In [52]:
ipo_processing['zip'] = ipo_processing.apply(lambda x: int(str(x['zip'])[:5]) if ( str(x['nation']) == 'United States' ) else 0 , axis=1)
ipo_processing = ipo_processing.merge(us_zip, how='left')
ipo_processing.state.fillna('Outside US', inplace=True)
#Remove ZIP.

columns_to_drop.append('zip')
#ipo_processing.drop(columns=['zip'],inplace = True)
len(ipo_processing)

3000

## 0.3 Pre-processing of Categorical Features


### issuer : done DROP ABoVE ???

issuer not relevant : ipo.naic_primary contient des fois des lettres? a supprimé?

In [53]:
len(ipo['issuer']) == len(ipo.issuer.unique())

True

In [54]:
columns_to_drop.append('issuer')

### ipo_1.naic_primary BBBBBA  contient des caractères je ne sais psa si c'est faux

In [55]:

to_dummies = ['description_cat','state','auditor','city','ind_group','veic_descr','naic_primary','naic_decr','public_descr','lockup_flag','sic_main','nation','lbo','prim_naic','prim_uop','pe_backed','shs_out_after','vc']

for col in to_dummies:
    print('processing column ' + col)
    ipo_processing,columns_to_drop = process_cat_columns(ipo_processing,col,columns_to_drop)

columns_with_n = ['mgrs_role','mgrs','ht_ind_gr','ht_ind','uop','legal','exch','br']
for col in columns_with_n:
    print('processing column ' + col)
    ipo_processing,columns_to_drop = process_categorical_with_sep(ipo_processing,col,'\n',columns_to_drop)

print('Processing column industry')
ipo_processing,columns_to_drop = process_categorical_with_sep(ipo_processing,'industry','/',columns_to_drop)



processing column description_cat
#nan :0
processing column state
#nan :0
processing column auditor
#nan :8
processing column city
#nan :6
processing column ind_group
#nan :0
processing column veic_descr
#nan :186
processing column naic_primary
#nan :0
processing column naic_decr
#nan :1
processing column public_descr
#nan :0
processing column lockup_flag
#nan :0
processing column sic_main
#nan :0
processing column nation
#nan :0
processing column lbo
#nan :0
processing column prim_naic
#nan :0
processing column prim_uop
#nan :0
processing column pe_backed
#nan :0
processing column shs_out_after
#nan :0
processing column vc
#nan :0
processing column mgrs_role
#nan :0
processing column mgrs
#nan :0
processing column ht_ind_gr
#nan :2
processing column ht_ind
#nan :2
processing column uop
#nan :0
processing column legal
#nan :0
processing column exch
#nan :0
processing column br
#nan :0
Processing column industry
#nan :0


## 0.4 Pre-processing of Ordinal Features

### price_range : Ordinal 
Since Price_range is an ordinal feature we will map it: 

In [56]:
ord_col = ['price_range']

ipo_processing.price_range.unique()

array(['Above range', 'Below range', 'Within range', nan], dtype=object)

In [57]:
price_range = ipo['price_range']

print(len(price_range.unique()))
price_range = replace_nan(price_range,'nan')
print(len(price_range.unique()))

dict_p = {'Above range': 3,
          'Within range':2,
          'Below range':1, 
            np.nan: 0}

ipo_processing.price_range = ipo_processing.price_range.replace(dict_p)

4
#nan :15
3


## 0.5 Pre-processing of Dates Features

### amd_date : Done
@Guillaume

In [58]:
#calculate number of amendment
ipo_processing['amd_nbr'] = ipo_processing['amd_date'].apply(lambda x:  str(x).count('\n')+1 if( str(x).count('\n') > 0) else ( 1 if(len(str(x)) > 0) else 0 ) )
num_col.append('amd_nbr')
columns_to_drop.append('amd_nbr')

### lockup_date, lockup_days : Done
@Guillaume

In [59]:
#Check number of missing values
print(ipo['lockup_days'].isna().sum())
print(ipo['lockup_date'].isna().sum())

#Check if missing values are the same
print(ipo.loc[ipo['lockup_days'].isna()].index)
print(ipo.loc[ipo['lockup_date'].isna()].index)

526
527
Int64Index([   7,    8,    9,   13,   18,   20,   23,   29,   37,   38,
            ...
            2946, 2953, 2959, 2963, 2964, 2970, 2971, 2987, 2988, 2989],
           dtype='int64', length=526)
Int64Index([   7,    8,    9,   13,   18,   20,   23,   29,   37,   38,
            ...
            2946, 2953, 2959, 2963, 2964, 2970, 2971, 2987, 2988, 2989],
           dtype='int64', length=527)


In [60]:
#Get days only last 
ipo_processing['lockup_days'] = ipo['lockup_days'].apply(lambda x: str(x) if( pd.isnull(x) ) else str(x)[-3:] )
#Cast it as a number
ipo_processing['lockup_days'] = ipo_processing['lockup_days'].apply(lambda x: int(x) if( str(x).isdigit() ) else np.nan )
#replace missing values
ipo_processing['lockup_days'] = replace_nan(ipo_processing['lockup_days'], '')

#ipo_processing['lockup_days'].head(50)

#nan :562


In [61]:
#We remove lockup_date because this information is rundandant in the lockup days we can calculate from the frist trade date
columns_to_drop.append('lockup_date')

### offer_date, first_trade_date, issue_date : DONE
@Guillaume

In [63]:
#Compare number of missing value offer_date and issue_date ares nearly the same.
print(ipo['issue_date'].isna().sum() )
print(ipo['offer_date'].isna().sum() )
print(ipo['first_trade_date'].isna().sum() )

0
0
281


In [62]:
#drop first_trade_date
columns_to_drop.append('first_trade_date')
columns_to_drop.append('issue_date')
columnWeTransform = 'offer_date'

In [64]:
# read dataset of the american 10 year bond rate
# This give us more info on the actual risk premium
rate = pd.read_csv('HQMCB10YR.csv')
rate.rename(columns = {'DATE':columnWeTransform, 'HQMCB10YR':'rate'}, inplace = True)
rate[columnWeTransform] = rate[columnWeTransform].apply(lambda x: pd.to_datetime(x))
rate.head(2)

,offer_date,rate
0,1984-01-01,12.89
1,1984-02-01,12.73


In [65]:
#Merge rate on the dataframe
ipo_processing[columnWeTransform] = ipo[columnWeTransform].apply(lambda x: pd.to_datetime(str(x.year)+'-'+str(x.month)+'-01') if( not(pd.isna(x))) else pd.to_datetime('1900-01-01'))
ipo_processing = ipo_processing.merge(rate, how='left')
columns_to_drop.append(columnWeTransform)
num_col.append('rate')

In [66]:
#Replace missing value rate
ipo_processing.rate = replace_nan(ipo_processing.rate, '')

#nan :0


In [67]:
#Replace missing value by the most frequent one.
ipo_processing[columnWeTransform] = ipo[columnWeTransform].apply(lambda x: str(x)[0:10])
ipo_processing[columnWeTransform] = replace_nan(ipo[columnWeTransform], '1900-01-01')
ipo_processing[columnWeTransform] = ipo_processing[columnWeTransform].apply(lambda x: pd.to_datetime(x))

#nan :0


In [68]:
#get datetime object for each row
ipo_processing[columnWeTransform+'_day'] = ipo_processing[columnWeTransform].apply(lambda x: int(x.day) )
cat_col.append(columnWeTransform+'_day')
ipo_processing[columnWeTransform+'_weekday'] = ipo_processing[columnWeTransform].apply(lambda x: int(x.weekday()) )
cat_col.append(columnWeTransform+'_weekday')
ipo_processing[columnWeTransform+'_month'] = ipo_processing[columnWeTransform].apply(lambda x: int(x.month) )
cat_col.append(columnWeTransform+'_month')
ipo_processing[columnWeTransform+'_year'] = ipo_processing[columnWeTransform].apply(lambda x: int(x.year) )
cat_col.append(columnWeTransform+'_year')
ipo_processing[columnWeTransform+'_timestamp'] = ipo_processing[columnWeTransform].apply(lambda x: dt.datetime(year=int(x.year), month=int(x.month), day=int(x.day)).timestamp())
num_col.append(columnWeTransform+'_timestamp')

#hot encode category
ipo_processing = pd.get_dummies(ipo_processing,  columns=[columnWeTransform+'_year',columnWeTransform+'_month',columnWeTransform+'_day',columnWeTransform+'_weekday'])

### date_amd : DONE
@Guillaume

In [70]:
#Convert from excel date to datetime
ipo_processing['date_amd'] = replace_nan(ipo['date_amd'], '')
ipo_processing['date_amd'] = ipo_processing['date_amd'].apply(lambda x: from_excel_ordinal(x) )
ipo_processing['date_amd'].head(1)

#nan :7


0   2014-09-22
Name: date_amd, dtype: datetime64[ns]

In [71]:
#Calculate difference between date_amd and offer_date save number of days as int
ipo_processing['offer_date'] = ipo['offer_date'].apply(lambda x: pd.to_datetime(x) )
ipo_processing['time_amd'] = ipo_processing.apply(lambda x: x['offer_date']-x['date_amd'], axis=1 ) #-x['offer_date'])
num_col.append('time_amd')
ipo_processing['time_amd'] = ipo_processing['time_amd'].apply(lambda x: x.days)

#column to drop:
columns_to_drop.append('date_amd')

Offering_Price: the price at which a company sells its shares to investors.

Num_Shares: the total number of outstanding shares.

Closing_Price: (at the end of the first day of training) price at which shares trade in the open market, measured at the end of the first day of trading.

## 0.6 Pre-processing of Numerical Features


We will start by numerical features that needs more preprocessing:

### Pre-processing

In [72]:
## sum over all values in round_tot
ipo_processing.round_tot = ipo_processing.round_tot.fillna(0)
ipo_processing.round_tot = ipo_processing.round_tot.replace(',','',regex = True)
ipo_processing.round_tot = ipo_processing.round_tot.apply(lambda x: x.split('\n') if str(x).isdigit()==False else [x] )

ipo_processing.round_tot = [sum([float(x) for x in j if re.match("^\d+?\.\d+?$", str(x)) or str(x).isdigit()])   for j in ipo_processing.round_tot.values]

In [73]:
ipo_processing['mgt_fee'] =ipo_processing.mgt_fee.replace('Comb.', np.nan)
ipo_processing['gross_spread'] = ipo_processing.gross_spread.replace('na', np.nan)


In [74]:
## replace ' in some numerical features:
col_to_replace = ['min_round_vexp','avg_round_vexp', 'max_round_vexp', 'min_firm_amt_vexp', 'avg_firm_amt_vexp', 'max_firm_amt_vexp',\
          'min_fund_amt_vexp', 'max_fund_amt_vexp']
for col in col_to_replace:
    ipo_processing[col] = ipo_processing[col].str.replace("'",'')

In [75]:
# Fill null values 
for col in num_col:
    ipo_processing[col] = DataFrameImputer().fit_transform(ipo_processing[[col]].astype(np.float32))[col]

In [77]:
floats = ipo.select_dtypes(include='float64')

for col in floats.columns:
    if not(col in columns_to_drop):
        print(col)
        ipo_processing[col] = DataFrameImputer().fit_transform(ipo_processing[[col]].astype(np.float32))[col]

Closing_Price
Offering_Price
pctchg_dj_1
pctchg_hp
pctchg_lp
pctchg_mp
pctchg_nasdaq_1
pctchg_sp_1
pctchg_sp_2
pctchg_dj_2
pctchg_nasdaq_2
pctchg_p
pctchg_sp_3
pctchg_sp_4
pctchg_amd_amt
pct_ins_shs_aft
pct_ins_shs_bef
mkt_cap
acc_fee
amd_hp
amd_lp
amd_mp
amd_pr_shs_pct
amd_pr_amt
amd_pr_shst_tot
amd_shst_tot
amt_filed
bluesky
book_proceeds
book_proceeds_ovt
bvps
comm_eq
comm_eq_bef
dj_avg_1
dj_avg_2
date_found
days_in_registr
deal_size
exp_pctofproceeds
exp_incl_gross
filing_date
free_float
gross_spread_allmkt
lastamd_tot
legal_exp
mktval_aft
mktval_bef
misc_exp
h_fil_p
l_fil_p
m_fil_p
nasdaq_avg_1
nasdaq_avg_2
num_emp
num_emp_date
shs_ourst_aft_prosp
ord_shs
prim_shs_1
prim_shs_2
prim_amt
prim_shs_3
print_exp
proceeds_1
proceeds_2
proceeds_3
SP1
SP2
SP3
SP4
SEC_fee
shs_out_bef
tang_ce_1
tang_ce_2
tor_ovt
reall_fee
tot_amt
tot_ass_after
tot ass_before
tot_inv
tot_mgtfee
underw_fee
num_rounds_vexp
num_firms_vexp
num_funds_vexp


In [76]:
for col in ints.columns:
     ipo_processing[col] = DataFrameImputer().fit_transform(ipo_processing[[col]].astype(np.float32))[col]
    

## 0.7 Pre-processing of Textual Features

### Risk_Factors :  Textual description of all the risk / done 


In [78]:
risks = ipo['Risk_Factors']

risks = process_text_columns(risks)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'stopwords' is not defined

### TFIDF

In [ ]:
# TFIDF --------------------------------------------------------------------------------------------
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 500)
risks_tfidf = vectorizer.fit_transform(risks)
risks_tfidf = risks_tfidf.toarray()

#Convert to a dataframe
risks_tfidf = pd.DataFrame(risks_tfidf)

In [ ]:
columnToDrop.append('Risk_Factors')

### Final check for the Nan



In [ ]:
len(ipo_processing)

In [ ]:
ipo_processing.isnull().values.sum()

In [ ]:
print(columnToDrop)

In [ ]:
ipo_processing.drop(columnToDrop,inplace=True,axis=1)

In [ ]:
ipo_processing.isnull().values.sum()

In [ ]:
import seaborn as sns

# Visualize pattern of missing data
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(ipo_processing.isnull(), cbar=False)

In [ ]:
ipo_processing.columns[ipo_processing.isna().any()].tolist()

In [ ]:
len(ipo_processing.columns)

## 0.9 : TARGET

In [ ]:
ipo_processing['Target'] = ipo_processing.Closing_Price >= ipo_processing.Offering_Price 

In [ ]:
len(ipo_processing.Closing_Price == ipo_processing.Offering_Price )

In [ ]:
len(ipo_processing['Target'])

In [ ]:
ipo_processing.Target = ipo_processing.Target.astype(int)

In [ ]:
counts = ipo_processing.Target.value_counts()

sns.set()

ones = counts[1]/len(counts) * 100
zeros = counts[0]/len(counts) * 100
labels = 'Price increased', 'Price decreased'
plt.pie( [ones, zeros], labels=labels, autopct='%1.2f%%', startangle=360)
plt.title('Distribution of target')
plt.show()

# Pour le preprocessing Je pense qu'il manque: Num_Shares, lockup_days, lockup_date Mais Num_shares est-ce qu'on l'utilise pour le target et donc pas pour les features?

## 0.7 Features Reduction

### 1. Check the correlation with the target
    -> Drop bottom 5%
### 2. Check the correlation between feature
    -> Drop 
    
### 0.7.1. Check the correlation with the target


In [ ]:
#Check type of values
ipo_processing.dtypes.unique()

In [ ]:
"""#Devide dataframe in function of type.
ints = ipo.select_dtypes(include='int64')
floats = ipo.select_dtypes(include='float64')
objects = ipo.select_dtypes(include='object')
timestamps = ipo.select_dtypes(include='M8[ns]')"""

print(ipo_processing.select_dtypes(include='object'))

In [ ]:
# Compute the correlation matrix
corr_dic = {}
for column in ipo_processing.columns:
    corr_dic[column] = abs(ipo_processing[column]
                           .corr(ipo_processing['Target']))

for w in sorted(corr_dic, key=corr_dic.get, reverse=True):
    print (w, corr_dic[w])

### 0.7.2. Check the correlation between feature


In [ ]:
import matplotlib.pyplot as plt

plt.matshow(dataframe.corr())

 _________________________________________________________ 
## Part 1

Predict whether the closing price is higher than the offering price using non-text fields. By non-text fields, we mean all fields except the 'Risk_Factors'. If the price goes up from opening to closing, assign a value of 1 to a new target variable called __Price_Increase__, otherwise assign 0.

    f(non-text-fields) -> Probability of being in class 1 

For the evaluation metric, report the area under the curve (AUC) and plot an ROC graph.

On doit peut être mettre tous les modèles dans un autre notebook si on va les utiliser dans les autres parties

In [ ]:
## J'enlève les dates en attendant
ipo_processing.drop( columns = ['lockup_date','lockup_days','Closing_Price','Offering_Price','first_trade_date','offer_date','issue_date'],inplace = True)

In [ ]:
target = ipo_processing['Target'] 
features = ipo_processing.drop(['Risk_Factors','Target'],axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

# Separate target and features into test and training and validation sets
seed = 1
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = seed)
X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = seed)

### Dummy classifier

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

bbaseline_clf = DummyClassifier(strategy='stratified', random_state = seed)

# Fit the dummy classifier 
bbaseline_clf.fit(X_train,y_train)

# Predict target probabilities of belonging to positive class
y_pred = bbaseline_clf.predict_proba(X_test)

# Compute area under the curve score
print('auc',roc_auc_score(y_test, y_pred[:,1]))

plot_roc_curve('Dummy',y_pred)

### Logistic Regression

In [ ]:
baseline_clf = LogisticRegression() 
baseline_clf.fit(X_train,y_train)
y_pred = baseline_clf.predict_proba(X_test)
print('Score ',roc_auc_score(y_test, y_pred[:,1]))
plot_roc_curve('Logistic Regression',y_pred)

### Logistic regression tuning c penalty 

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Standardize features and classifier in a single pipeline
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('lr_clf', LogisticRegression()))
pipeline = Pipeline(estimators)
pipeline.set_params(lr_clf__penalty='l1')

# Finding best value of C using validation set
scores = []
Cs = []
for C in np.logspace(-4, 5, 10):
    pipeline.set_params(lr_clf__C=C) 
    pipeline.fit(X_train,y_train)
    y_train_pred = pipeline.predict_proba(X_test)
    scores.append(roc_auc_score(y_test, y_train_pred[:,1]))
    Cs.append(C)

best_C = Cs[scores.index(max(scores))]
print ('best C = %d with auc score = %2.4f' %(best_C, max(scores)))

# Performance of the tuned model on test set
pipeline.set_params(lr_clf__C=best_C)
pipeline.fit(X_train,y_train)
y_pred_lr = pipeline.predict_proba(X_test)
score = roc_auc_score(y_test, y_pred_lr[:,1])
print ('lr classifer auc with l1 regularization = %2.4f' %score)
plot_roc_curve('l1 regularization',y_pred_lr)

### KNN tuning N neighbors

In [ ]:
# Standardize features and classifier in a single pipeline
from sklearn.neighbors import KNeighborsClassifier

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('knn_clf', KNeighborsClassifier()))
pipeline = Pipeline(estimators)

# Finding best value of K using validation set
scores = []
Ks = []
for K in [int(i) for i in np.linspace(5, 95, 10)]:
    pipeline.set_params(knn_clf__n_neighbors = K) 
    pipeline.fit(X_train_train,y_train_train)
    y_train_pred = pipeline.predict_proba(X_train_val)
    scores.append(roc_auc_score(y_train_val, y_train_pred[:,1]))
    Ks.append(K)

best_K = Ks[scores.index(max(scores))]
print ('best K = %d with auc score = %2.4f' %(best_K, max(scores)))

# Performance of the tuned model on test set
pipeline.set_params(knn_clf__n_neighbors = best_K)
pipeline.fit(X_train,y_train)
y_pred_knn = pipeline.predict_proba(X_test)
score = roc_auc_score(y_test, y_pred_knn[:,1])
print ('knn classifer auc = %2.4f' %score)
plot_roc_curve('KNN',y_pred_knn)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define a random classifier pipeline
estimators = []
estimators.append(('rf_clf', RandomForestClassifier()))
pipeline = Pipeline(estimators)
pipeline.set_params(rf_clf__random_state = seed)
    
# Finding best value of n_estimators using validation set
scores = []
NSs = []
for NS in [int(i) for i in np.linspace(10, 100, 10)]:
    pipeline.set_params(rf_clf__n_estimators = NS) 
    pipeline.fit(X_train_train,y_train_train)
    y_train_pred = pipeline.predict_proba(X_train_val)
    scores.append(roc_auc_score(y_train_val, y_train_pred[:,1]))
    NSs.append(NS)

best_NS = NSs[scores.index(max(scores))]
print ('best NS = %d with auc score = %2.4f' %(best_NS, max(scores)))

# Performance of the tuned model on test set
pipeline.set_params(rf_clf__n_estimators = best_NS)
pipeline.fit(X_train,y_train)
y_pred_rf = pipeline.predict_proba(X_test)
score = roc_auc_score(y_test, y_pred_rf[:,1])
print ('rf classifer auc = %2.4f' %score)

In [ ]:
plot_roc_curve('KNN',y_pred_rf)

###  Gradient Boosting classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Define a random classifier pipeline
estimators = []
estimators.append(('gb_clf', GradientBoostingClassifier()))
pipeline = Pipeline(estimators)
pipeline.set_params(gb_clf__random_state = seed)
    
# Finding best value of n_estimators using validation set
scores = []
NSs = []
for NS in [int(i) for i in np.linspace(10, 100, 10)]:
    pipeline.set_params(gb_clf__n_estimators = NS) 
    pipeline.fit(X_train_train,y_train_train)
    y_train_pred = pipeline.predict_proba(X_train_val)
    scores.append(roc_auc_score(y_train_val, y_train_pred[:,1]))
    NSs.append(NS)

best_NS = NSs[scores.index(max(scores))]
print ('best NS = %d with auc score = %2.4f' %(best_NS, max(scores)))

# Performance of the tuned model on test set
pipeline.set_params(gb_clf__n_estimators = best_NS)
pipeline.fit(X_train,y_train)
y_pred_gb = pipeline.predict_proba(X_test)
score = roc_auc_score(y_test, y_pred_gb[:,1])
plot_roc_curve('GradientBoostingClassifier',y_pred_gb)

In [ ]:
print(score)

## Part 2

Predict whether the closing price is higher than the offering price using __only__ textual field 'Risk_Factors'. If the price goes up from opening to closing, assign a value of 1 to a new target variable called __Price_Increase__, otherwise assign 0.

    f(text-fields) -> Probability of being in class 1 

For the evaluation metric, report the area under the curve (AUC) and plot an ROC graph.

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

def nested_cv(X, y, est_pipe, p_grid, p_score, n_splits_inner = 3, n_splits_outer = 3, n_cores = 1, seed = 0):

    # Cross-validation schema for inner and outer loops (stratified if it is a classification)
    inner_cv = KFold(n_splits = n_splits_inner, shuffle = True, random_state = seed)
    outer_cv = KFold(n_splits = n_splits_outer, shuffle = True, random_state = seed)
    
    # Grid search to tune hyper parameters
    est = GridSearchCV(estimator = est_pipe, param_grid = p_grid, cv = inner_cv, scoring = p_score, n_jobs = n_cores)

    # Nested CV with parameter optimization
    nested_scores = cross_val_score(estimator = est, X = X, y = y, cv = outer_cv, scoring = p_score, n_jobs = n_cores)
    
    print('Average score: %0.4f (+/- %0.4f)' % (nested_scores.mean(), nested_scores.std() * 1.96))

In [ ]:
# Dataframe of Risk Factors :  risks_tfidf    
features = risks_tfidf
seed = 0

warnings.filterwarnings('ignore')
seed = 0

# Define pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

estimators = []
estimators.append(('rf_clf', RandomForestClassifier()))
rf_pipe = Pipeline(estimators)
rf_pipe.set_params(rf_clf__random_state = seed)

# Fixed parameters
score = 'accuracy'

# Setup possible values of parameters to optimize over
p_grid = {"rf_clf__n_estimators": [int(i) for i in np.linspace(10.0, 50.0, 5)]}

nested_cv(X = features, y = target, est_pipe = rf_pipe, p_grid = p_grid, p_score = score, n_cores = -1)

### PCA

In [ ]:
# Regroupe to 50 Fields 
from sklearn.decomposition import PCA

pca = PCA(n_components=100)

principalComponents = pca.fit_transform(risks_tfidf)
principalComponents = pd.DataFrame(principalComponents)



#Update the name of the columns
# get length of df's columns
num_cols = 100
# generate range of ints for suffixes
rng = range(0,num_cols)

new_cols = [ 'risk_'+str(i) for i in rng]

# ensure the length of the new columns list is equal to the length of df's columns
principalComponents.columns = new_cols[:num_cols]

principalComponents.head()

In [ ]:
features = principalComponents
seed = 0

warnings.filterwarnings('ignore')
seed = 0

# Define pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

estimators = []
estimators.append(('rf_clf', RandomForestClassifier()))
rf_pipe = Pipeline(estimators)
rf_pipe.set_params(rf_clf__random_state = seed)

# Fixed parameters
score = 'accuracy'

# Setup possible values of parameters to optimize over
p_grid = {"rf_clf__n_estimators": [int(i) for i in [10, 20, 50, 100]]}

nested_cv(X = features, y = target, est_pipe = rf_pipe, p_grid = p_grid, p_score = score, n_cores = -1)


## Word Vectors : Word2Vec

In [ ]:

import nltk.data
import nltk
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def review_to_wordlist( review ):
    
    review_text = BeautifulSoup(review).get_text()
   
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    words = review_text.lower().split()
    
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    
    stemmer = SnowballStemmer('english')
    words = [stemmer.stem(w) for w in words]
    
    return(words)

In [ ]:
# Define a function to split a review into parsed sentences, where each sentence is a word list

def review_to_sentences( review, tokenizer ):
    
    raw_sentences = tokenizer.tokenize(review.strip())  
    sentences = []
    for raw_sentence in raw_sentences:      
        if len(raw_sentence) > 0:           
            sentences.append( review_to_wordlist( raw_sentence ))
   
    return sentences

In [ ]:

sentences = []
risks = ipo['Risk_Factors']
for risk in risks:
    sentences += review_to_sentences(risk, tokenizer)

In [ ]:
# Train word vectors

warnings.filterwarnings('ignore')

# Set values for various parameters
num_features = 300    # word vector dimensionality                      
min_word_count = 40   # minimum word count                        
num_workers = 16      # number of threads to run in parallel
context = 10          # context window size                                                                                    

# Initialize and train the model 
from gensim.models import word2vec
print ('Training model...')
w2v_model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)
print ('Done !')

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient
w2v_model.init_sims(replace=True)
 

In [ ]:
# Function to average all of the word vectors in a given paragraph

def makeFeatureVec(words, model, num_features):
    
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,), dtype='float32')
    nwords = 0.
     
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [ ]:

# Given a set of reviews (each one a list of words), calculate 
# the average feature vector for each one and return a 2D numpy array

def getAvgFeatureVecs(reviews, model, num_features):
    
    # Initialize a counter
    counter = 0
    
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype='float32')
     
    # Loop through the reviews
    for review in reviews:
       
       # Print a status message every 1000th review
       if counter%1000 == 0:
           print ('Review %d of %d' % (counter, len(reviews)))
       
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [ ]:
# Calculate average feature vectors for review data,
# using the functions we defined above.

clean_data_reviews = []
for review in ipo['Risk_Factors']:
    clean_data_reviews.append( review_to_wordlist( review ))

w2v_features = getAvgFeatureVecs( clean_data_reviews, w2v_model, num_features )

In [ ]:

%%time
warnings.filterwarnings('ignore')
seed = 0

# Define pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

estimators = []
estimators.append(('rf_clf', RandomForestClassifier()))
rf_pipe = Pipeline(estimators)
rf_pipe.set_params(rf_clf__random_state = seed)

# Fixed parameters
score = 'accuracy'

# Setup possible values of parameters to optimize over
p_grid = {"rf_clf__n_estimators": [int(i) for i in np.linspace(10.0, 50.0, 5)]}

nested_cv(X = w2v_features, y = target, est_pipe = rf_pipe, p_grid = p_grid, p_score = score, n_cores = -1)


### Classification using Random Forest with Average Word2Vec Features

In [ ]:
%%time
warnings.filterwarnings('ignore')
seed = 0

# Define pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

estimators = []
estimators.append(('rf_clf', RandomForestClassifier()))
rf_pipe = Pipeline(estimators)
rf_pipe.set_params(rf_clf__random_state = seed)

# Fixed parameters
score = 'accuracy'

# Setup possible values of parameters to optimize over
p_grid = {"rf_clf__n_estimators": [int(i) for i in np.linspace(10.0, 50.0, 5)]}

nested_cv(X = w2v_features, y = target, est_pipe = rf_pipe, p_grid = p_grid, p_score = score, n_cores = -1)

## Paragraph vectors : Doc2Vec


In [ ]:
# Doc2Vec needs each review to be tagged with some sort of ids
# Here we tag each review with the 'id' field

import gensim

tagged_clean_data_reviews = []
for uid, review in zip(data['id'], clean_data_reviews):
    tagged_clean_data_reviews.append(gensim.models.doc2vec.TaggedDocument(words=review, tags=['%s' % uid[1:-1]]))

## Part 3

Predict whether the closing price is higher than the offering price using __all__ fields. If the price goes up from opening to closing, assign a value of 1 to a new target variable called __Price_Increase__, otherwise assign 0.

    f(all-fields) -> Probability of being in class 1 
    
For the evaluation metric, report the area under the curve (AUC) and plot an ROC graph.

## Part 4
  
Predict the share price at the end of the day using __all__ fields.

    f(all-fields) ->  Share price at the end of the first day of trading
    
For the evaluation metric, report statisitcs for R-squared, Residual Mean Squared Error, Mean Absolute Error, and Median Absolute Error; however, be sure to tune and hypertune your models using R-Squared.

## Data

As mentioned earlier, you can find the dataset under project directory in the course git repository under the name of *ipo.xlsx*. The description of each variable can also be found in *variable_description.xlsx*.

## Requirements

We expect your solution for each step to contain the following:

* data preprocessing and feature extraction (can be shared across different steps)
* feature reduction
* train, tune and test different predictive models
* model comparison and arguing about the best model (don't forget mentioning a baseline model)
* predict the labels of the to_predict dataset using your final model
* discussion on possible additional tasks that can be done to boost the performance

## Deliverables

* Predict values from your *best* predictive model for the target variable in Parts 1 to 4 above, and insert those values into the file __*ipo_to_predict.xlsx*__ The fields to be completed by you are: __Price_Change_Non_Textual__ (Part 1), __Price_Change_Textual__ (Part 2), __Price_Change_All__ (Part 3), and __Price_All__ (Part 4).  
  
  
* Deliver a Jupyter notebook with an explanation of your methods, codes and results. Don't forget to divide your notebook into different parts, which clearly shows your solution to the common pre-processing as well as different steps separately. 
    
    

* Submit your final notebook and files into the git repository of the team (we will create that git repo for you).


* Present your results in the final session of the course. Communicate them in a clear and concise manner. The goal is to learn how to present your results to stakeholders at the right level of detail. **We will discuss this more in classe**. 

## Tips

* Take some time at the start of the project to educate yourslef about the IPO process. We privde you wiht two main texts in the class repository under *resources* folder. Understanding how variables relate to the target outcomes will help you to construct new measures from the tabularized data and/or selecting or eliminating features that relate to the target variable.  


* Present your results as a story - this is very important!   
  

* Document all of your assumptions (e.g. evaluation metric, hyper-parameter values, ...).  


* Make sure your code will run and results are reproducible (fix random seeds, etc.).  


*  Comment your blocks of code (and lines of code if needed) and anything in your story/logic that might not be obvious by looking at your code.    


* To speed up experimentation, you might use a small sample of the original dataset to do your initial coding. Also try to use all possible cores for computation, by setting the option of n_jobs = -1, when needed. 


* Try to be creative to improve your predictions, but don't forget that it is also important to explain your line of thinking/reasoning.


* Your final grade is based on the whole process of doing the project and not just based on your results on the unseen data. 

## Grading

Grading of the project (apart from presentation), is based on the following components:
    
    20 %  Documentation and organization of your notebook
    15 %  Quality and commenting of code
    10 %  Pre-processing
    15 %  Part 1
    15 %  Part 2
    10 %  Part 3
    15 %  Part 4
    
     5%   Bonus Contest


## Bonus Contest

As an optional bonus contest at the end of the project, we will award an extra 5% of the total project grade to the team that comes up with the "best" strategy for investing into IPOs based on your estimated model(s). Specifically, assume you have USD 1,000,000 to invest into the IPO stocks that appear in the "unseen" file __*ipo_to_predict.xlsx*__. In the column "Your_Bet", allocate some portion of that USD 1,000,000 to each of the stocks listed in the unseen file. The total allocation must sum to $1,000,000. The top team making the most money (once outcomes are revealed at time of grading) will earn the 5% bonus. 